In [364]:
import pandas as pd

In [365]:
df_enc = pd.read_csv('data\encounter-events.csv')
df_loit = pd.read_csv('data\loitering-events.csv')
df_ves = pd.read_csv('data/transshipment-vessels.csv')

# Data Transformation

## Preprocessing

### Encounter Dataset

This code:

* removes all entries with duplicate values over all features
* removes outliers in duration_hr column
* transforms timestamps into floats and creates new time features as datetime objects
* adds a support feature to the dataset

In [366]:
df_enc_mod = df_enc.copy()

df_enc_mod = df_enc_mod.loc[df_enc_mod['duration_hr'] <= 65]  # remove outliers detected in the EDA file

df_enc_mod.drop_duplicates(inplace = True)  # remove duplicates

print('{} encounterings left in the dataset'.format(df_enc_mod.shape[0]))

11470 encounterings left in the dataset


In [367]:
df_enc_mod['start_time'] = pd.to_datetime(df_enc['start_time'])  # parse time objects
df_enc_mod['end_time'] = pd.to_datetime(df_enc['end_time'])

df_enc_mod['starting_timestamp'] = df_enc_mod['start_time'].apply(lambda x: x.timestamp()) # Changing format of timestamps from object to float
df_enc_mod['ending_timestamp'] = df_enc_mod['end_time'].apply(lambda x: x.timestamp())
df_enc_mod.head()

,fishing_vessel_mmsi,transshipment_vessel_mmsi,start_time,end_time,mean_latitude,mean_longitude,duration_hr,median_distance_km,median_speed_knots,starting_timestamp,ending_timestamp
0,416565000,354240000,2016-11-18 14:30:00+00:00,2016-11-19 01:50:00+00:00,-17.039085,-79.063725,11.333333,0.038188,0.585402,1.479479e+09,1.479520e+09
1,412679190,354240000,2016-12-11 14:50:00+00:00,2016-12-11 19:50:00+00:00,-20.269608,-79.244953,5.000000,0.020033,0.575663,1.481468e+09,1.481486e+09
2,440863000,354240000,2017-06-13 12:50:00+00:00,2017-06-15 01:20:00+00:00,-62.640767,-60.690240,36.500000,0.054992,0.019775,1.497358e+09,1.497490e+09
3,416563000,354240000,2016-11-15 11:30:00+00:00,2016-11-16 04:00:00+00:00,-17.046586,-79.061923,16.500000,0.036427,1.023917,1.479209e+09,1.479269e+09
4,441309000,354240000,2017-05-19 00:40:00+00:00,2017-05-19 20:50:00+00:00,-46.627878,-60.554922,20.166667,0.034053,0.544031,1.495154e+09,1.495227e+09


In [368]:
df_enc_mod['from_encounter'] = 1  # support variable need later after merging

In [369]:
df_enc_mod.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11470 entries, 0 to 11680
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype              
---  ------                     --------------  -----              
 0   fishing_vessel_mmsi        11470 non-null  int64              
 1   transshipment_vessel_mmsi  11470 non-null  int64              
 2   start_time                 11470 non-null  datetime64[ns, UTC]
 3   end_time                   11470 non-null  datetime64[ns, UTC]
 4   mean_latitude              11470 non-null  float64            
 5   mean_longitude             11470 non-null  float64            
 6   duration_hr                11470 non-null  float64            
 7   median_distance_km         11470 non-null  float64            
 8   median_speed_knots         11470 non-null  float64            
 9   starting_timestamp         11470 non-null  float64            
 10  ending_timestamp           11470 non-null  float64            
 11  fr

### Loitering Dataset

This code:

* removes all entries with duplicate values over all features
* remove outliers in total_even_duration column
* Creates new feature and removes redundant features
* transforms timestamps into floats and creates new time features as datetime objects
* adds a support feature to the dataset
* both transforms and renames features to those in the encounter dataset

In [370]:
df_loit_mod =  df_loit.copy()

df_loit_mod = df_loit_mod.loc[df_loit_mod['total_event_duration'] <= 200]  # remove outliers detected in the EDA file

df_loit_mod.drop_duplicates(inplace = True)  # dropping duplicates

print('{} loitering events left in the dataset'.format(df_loit_mod.shape[0]))

45919 loitering events left in the dataset


The next step consists of creating mean variable for both latitude and longitude coordinates to match the format of the encounter dataset

In [371]:
df_loit_mod['mean_latitude'] = (df_loit_mod['starting_latitude']+df_loit_mod['ending_latitude'])/2  # transforming locations to mean
df_loit_mod['mean_longitude'] = (df_loit_mod['starting_longitude']+df_loit_mod['ending_longitude'])/2
df_loit_mod.drop(columns=['starting_latitude','ending_latitude','starting_longitude', 'ending_longitude'], inplace=True)  # drop old columns

In [372]:
df_loit_mod['start_time'] = pd.to_datetime(df_loit['starting_timestamp'])  # Create new start time column with datetime format
df_loit_mod['end_time'] = pd.to_datetime(df_loit['ending_timestamp'])

df_loit_mod['starting_timestamp'] = df_loit_mod['start_time'].apply(lambda x: x.timestamp()) # Changing format of timestamps from object to float
df_loit_mod['ending_timestamp'] = df_loit_mod['end_time'].apply(lambda x: x.timestamp())
df_loit_mod.head()

,transshipment_mmsi,starting_timestamp,ending_timestamp,median_speed_knots,total_event_duration,mean_latitude,mean_longitude,start_time,end_time
0,256064000,1.504664e+09,1.504713e+09,1.091065,13.915556,12.760875,-69.621178,2017-09-06 02:11:32+00:00,2017-09-06 15:44:15+00:00
1,256064000,1.500730e+09,1.500768e+09,1.548335,10.897639,25.399182,-56.395525,2017-07-22 13:19:49+00:00,2017-07-22 23:59:17+00:00
2,256064000,1.511967e+09,1.512028e+09,1.503456,18.001389,10.588957,-78.698550,2017-11-29 14:48:51+00:00,2017-11-30 07:43:36+00:00
3,256064000,1.504828e+09,1.504883e+09,0.650355,16.193611,11.933280,-75.544505,2017-09-07 23:39:03+00:00,2017-09-08 14:59:26+00:00
4,256064000,1.503012e+09,1.503079e+09,1.009874,19.748333,45.957853,-23.623173,2017-08-17 23:18:21+00:00,2017-08-18 17:56:30+00:00


We rename the transshipment column make clear that it's a transshipment VESSEL attribute not one of a possible transshipment itself
and rename the duration column to match the format of the encounter data set

In [373]:
df_loit_mod.rename(columns ={'transshipment_mmsi':'transshipment_vessel_mmsi', 'total_event_duration':'duration_hr'}, inplace = True)

df_loit_mod['from_encounter'] = 0  # support variable need later after merging

In [374]:
df_loit_mod.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45919 entries, 0 to 46568
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype              
---  ------                     --------------  -----              
 0   transshipment_vessel_mmsi  45919 non-null  int64              
 1   starting_timestamp         45919 non-null  float64            
 2   ending_timestamp           45919 non-null  float64            
 3   median_speed_knots         45919 non-null  float64            
 4   duration_hr                45919 non-null  float64            
 5   mean_latitude              45919 non-null  float64            
 6   mean_longitude             45919 non-null  float64            
 7   start_time                 45919 non-null  datetime64[ns, UTC]
 8   end_time                   45919 non-null  datetime64[ns, UTC]
 9   from_encounter             45919 non-null  int64              
dtypes: datetime64[ns, UTC](2), float64(6), int64(2)
memory usage: 3.9 MB


### Vessel Dataset

This code:

* removes all entries with duplicate values in mmsi [Team Decision] to avoid issues with future joins over all datasets
* removes all entries with NA or null values in the mmsi feature
* transforms timestamps into datetime objects

In [375]:
df_ves.loc[df_ves['mmsi'].duplicated(keep=False)].head()

,mmsi,shipname,callsign,flag,imo,first_timestamp,last_timestamp
17,352610000.0,HAINANBAOSHA12,C6NP6,BHS,9063653.0,2013-03-26T11:30:49Z,2013-03-30T10:46:25Z
46,312413000.0,QIAO TAI,V3TV5,BLZ,9677595.0,2012-12-23T04:24:29Z,2014-09-22T06:31:57Z
50,518332000.0,TOKACHIFROST,V3RS6,BLZ,8503242.0,2015-05-15T17:57:20Z,2015-12-11T10:45:52Z
71,412441730.0,HAI FENG 101,BZ2UN,CHN,8785448.0,2012-08-25T02:10:16Z,2018-04-16T06:21:11Z
74,412441730.0,HAIFENG101,NaN,CHN,NaN,2012-08-25T02:10:16Z,2018-04-16T06:21:11Z


In [376]:
df_ves_mod = df_ves.copy()

df_ves_mod.dropna(subset=['mmsi'], inplace=True)
# this is necessary to avoid weird (non sql) behavior from merge (nulls are recognized as keys!)

df_ves_mod.drop_duplicates(subset = ['mmsi'], inplace = True)  # dropping duplicates in feature mmsi

For the dates, we want to have both times in datetime format and timestamps in float format

In [377]:
df_ves_mod['first_time'] = pd.to_datetime(df_ves_mod['first_timestamp'])  # Create new first time column with datetime format
df_ves_mod['last_time'] = pd.to_datetime(df_ves_mod['last_timestamp'])

df_ves_mod['first_timestamp'] = df_ves_mod['first_time'].apply(lambda x: x.timestamp())  # Change format from object to float
df_ves_mod['last_timestamp'] = df_ves_mod['last_time'].apply(lambda x: x.timestamp())

In [378]:
df_ves_mod.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1039 entries, 0 to 1069
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype              
---  ------           --------------  -----              
 0   mmsi             1039 non-null   float64            
 1   shipname         1038 non-null   object             
 2   callsign         1029 non-null   object             
 3   flag             1039 non-null   object             
 4   imo              991 non-null    float64            
 5   first_timestamp  1039 non-null   float64            
 6   last_timestamp   1039 non-null   float64            
 7   first_time       1039 non-null   datetime64[ns, UTC]
 8   last_time        1039 non-null   datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](2), float64(4), object(3)
memory usage: 81.2+ KB


## Aggregation

This code:

* Aggregates(Stacks) Encounter and Loitering Datasets by keys and adds non matching features as additional columns
* Reinserts the target feature as the first column 

In [379]:
df_agg = pd.concat([df_enc_mod,df_loit_mod], ignore_index=True)

#setting support variable as first column:
df_agg.insert(0, 'from_encounter',df_agg.pop('from_encounter'))

df_agg.info()
df_agg.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57389 entries, 0 to 57388
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype              
---  ------                     --------------  -----              
 0   from_encounter             57389 non-null  int64              
 1   fishing_vessel_mmsi        11470 non-null  float64            
 2   transshipment_vessel_mmsi  57389 non-null  int64              
 3   start_time                 57389 non-null  datetime64[ns, UTC]
 4   end_time                   57389 non-null  datetime64[ns, UTC]
 5   mean_latitude              57389 non-null  float64            
 6   mean_longitude             57389 non-null  float64            
 7   duration_hr                57389 non-null  float64            
 8   median_distance_km         11470 non-null  float64            
 9   median_speed_knots         57389 non-null  float64            
 10  starting_timestamp         57389 non-null  float64            
 11  en

,from_encounter,fishing_vessel_mmsi,transshipment_vessel_mmsi,start_time,end_time,mean_latitude,mean_longitude,duration_hr,median_distance_km,median_speed_knots,starting_timestamp,ending_timestamp
0,1,416565000.0,354240000,2016-11-18 14:30:00+00:00,2016-11-19 01:50:00+00:00,-17.039085,-79.063725,11.333333,0.038188,0.585402,1.479479e+09,1.479520e+09
1,1,412679190.0,354240000,2016-12-11 14:50:00+00:00,2016-12-11 19:50:00+00:00,-20.269608,-79.244953,5.000000,0.020033,0.575663,1.481468e+09,1.481486e+09
2,1,440863000.0,354240000,2017-06-13 12:50:00+00:00,2017-06-15 01:20:00+00:00,-62.640767,-60.690240,36.500000,0.054992,0.019775,1.497358e+09,1.497490e+09
3,1,416563000.0,354240000,2016-11-15 11:30:00+00:00,2016-11-16 04:00:00+00:00,-17.046586,-79.061923,16.500000,0.036427,1.023917,1.479209e+09,1.479269e+09
4,1,441309000.0,354240000,2017-05-19 00:40:00+00:00,2017-05-19 20:50:00+00:00,-46.627878,-60.554922,20.166667,0.034053,0.544031,1.495154e+09,1.495227e+09


##  Merging

This code:

* Merges the aggregated datasets with the vessel data on the key pair 'transshipment_vessel_mmsi' and 'mmsi'
* This is sufficient because no fishing vessels are in the vessel dataset
* manually adds a suffix of '_t' to all features originating from the vessel dataset


In [380]:
df_clean_all = pd.merge(df_agg, df_ves_mod, how ='left', left_on ='transshipment_vessel_mmsi', right_on='mmsi')


####Optional: add suffix to all added variables
df_clean_all.rename(columns= { 'mmsi' : 'mmsi_t', 'shipname': 'shipname_t', 'callsign':'callsign_t', 'flag':'flag_t', 'imo': 'imo_t',\
     'first_timestamp': 'first_timestamp_t', 'last_timestamp':'last_timestamp_t'}, inplace = True)

####This could be use to merge on fishing vessel, this is however not necessary in this context(No fishing vessels, only tranship)

# df_clean_all = pd.merge(df_clean_all, df_ves_mod, how ='left', left_on ='fishing_vessel_mmsi', right_on='mmsi')
# df_clean_all.rename(columns= { 'mmsi' : 'mmsi_f', 'shipname': 'shipname_f', 'callsign':'callsign_f', 'flag':'flag_f', 'imo': 'imo_f',\
#      'first_timestamp': 'first_timestamp_f', 'last_timestamp':'last_timestamp_f'}, inplace = True)

df_clean_all.info()
df_clean_all.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57389 entries, 0 to 57388
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype              
---  ------                     --------------  -----              
 0   from_encounter             57389 non-null  int64              
 1   fishing_vessel_mmsi        11470 non-null  float64            
 2   transshipment_vessel_mmsi  57389 non-null  int64              
 3   start_time                 57389 non-null  datetime64[ns, UTC]
 4   end_time                   57389 non-null  datetime64[ns, UTC]
 5   mean_latitude              57389 non-null  float64            
 6   mean_longitude             57389 non-null  float64            
 7   duration_hr                57389 non-null  float64            
 8   median_distance_km         11470 non-null  float64            
 9   median_speed_knots         57389 non-null  float64            
 10  starting_timestamp         57389 non-null  float64            
 11  en

,from_encounter,fishing_vessel_mmsi,transshipment_vessel_mmsi,start_time,end_time,mean_latitude,mean_longitude,duration_hr,median_distance_km,median_speed_knots,...,ending_timestamp,mmsi_t,shipname_t,callsign_t,flag_t,imo_t,first_timestamp_t,last_timestamp_t,first_time,last_time
0,1,416565000.0,354240000,2016-11-18 14:30:00+00:00,2016-11-19 01:50:00+00:00,-17.039085,-79.063725,11.333333,0.038188,0.585402,...,1.479520e+09,354240000.0,CRYSTAL REEFER,3FFY5,PAN,9017276.0,1.422748e+09,1.489109e+09,2015-01-31 23:43:15+00:00,2017-03-10 01:21:55+00:00
1,1,412679190.0,354240000,2016-12-11 14:50:00+00:00,2016-12-11 19:50:00+00:00,-20.269608,-79.244953,5.000000,0.020033,0.575663,...,1.481486e+09,354240000.0,CRYSTAL REEFER,3FFY5,PAN,9017276.0,1.422748e+09,1.489109e+09,2015-01-31 23:43:15+00:00,2017-03-10 01:21:55+00:00
2,1,440863000.0,354240000,2017-06-13 12:50:00+00:00,2017-06-15 01:20:00+00:00,-62.640767,-60.690240,36.500000,0.054992,0.019775,...,1.497490e+09,354240000.0,CRYSTAL REEFER,3FFY5,PAN,9017276.0,1.422748e+09,1.489109e+09,2015-01-31 23:43:15+00:00,2017-03-10 01:21:55+00:00
3,1,416563000.0,354240000,2016-11-15 11:30:00+00:00,2016-11-16 04:00:00+00:00,-17.046586,-79.061923,16.500000,0.036427,1.023917,...,1.479269e+09,354240000.0,CRYSTAL REEFER,3FFY5,PAN,9017276.0,1.422748e+09,1.489109e+09,2015-01-31 23:43:15+00:00,2017-03-10 01:21:55+00:00
4,1,441309000.0,354240000,2017-05-19 00:40:00+00:00,2017-05-19 20:50:00+00:00,-46.627878,-60.554922,20.166667,0.034053,0.544031,...,1.495227e+09,354240000.0,CRYSTAL REEFER,3FFY5,PAN,9017276.0,1.422748e+09,1.489109e+09,2015-01-31 23:43:15+00:00,2017-03-10 01:21:55+00:00


In [381]:
df_clean_all

,from_encounter,fishing_vessel_mmsi,transshipment_vessel_mmsi,start_time,end_time,mean_latitude,mean_longitude,duration_hr,median_distance_km,median_speed_knots,...,ending_timestamp,mmsi_t,shipname_t,callsign_t,flag_t,imo_t,first_timestamp_t,last_timestamp_t,first_time,last_time
0,1,416565000.0,354240000,2016-11-18 14:30:00+00:00,2016-11-19 01:50:00+00:00,-17.039085,-79.063725,11.333333,0.038188,0.585402,...,1.479520e+09,354240000.0,CRYSTAL REEFER,3FFY5,PAN,9017276.0,1.422748e+09,1.489109e+09,2015-01-31 23:43:15+00:00,2017-03-10 01:21:55+00:00
1,1,412679190.0,354240000,2016-12-11 14:50:00+00:00,2016-12-11 19:50:00+00:00,-20.269608,-79.244953,5.000000,0.020033,0.575663,...,1.481486e+09,354240000.0,CRYSTAL REEFER,3FFY5,PAN,9017276.0,1.422748e+09,1.489109e+09,2015-01-31 23:43:15+00:00,2017-03-10 01:21:55+00:00
2,1,440863000.0,354240000,2017-06-13 12:50:00+00:00,2017-06-15 01:20:00+00:00,-62.640767,-60.690240,36.500000,0.054992,0.019775,...,1.497490e+09,354240000.0,CRYSTAL REEFER,3FFY5,PAN,9017276.0,1.422748e+09,1.489109e+09,2015-01-31 23:43:15+00:00,2017-03-10 01:21:55+00:00
3,1,416563000.0,354240000,2016-11-15 11:30:00+00:00,2016-11-16 04:00:00+00:00,-17.046586,-79.061923,16.500000,0.036427,1.023917,...,1.479269e+09,354240000.0,CRYSTAL REEFER,3FFY5,PAN,9017276.0,1.422748e+09,1.489109e+09,2015-01-31 23:43:15+00:00,2017-03-10 01:21:55+00:00
4,1,441309000.0,354240000,2017-05-19 00:40:00+00:00,2017-05-19 20:50:00+00:00,-46.627878,-60.554922,20.166667,0.034053,0.544031,...,1.495227e+09,354240000.0,CRYSTAL REEFER,3FFY5,PAN,9017276.0,1.422748e+09,1.489109e+09,2015-01-31 23:43:15+00:00,2017-03-10 01:21:55+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57384,0,NaN,273349630,2017-09-29 07:55:39+00:00,2017-09-29 22:49:11+00:00,51.945812,155.562330,16.982917,NaN,1.092219,...,1.506725e+09,273349630.0,ZODIAK,UBFG9,RUS,8712300.0,1.328268e+09,1.530318e+09,2012-02-03 11:26:34+00:00,2018-06-30 00:13:21+00:00
57385,0,NaN,273349630,2016-11-19 08:08:13+00:00,2016-11-20 00:58:25+00:00,48.793993,141.242218,17.053056,NaN,0.764958,...,1.479604e+09,273349630.0,ZODIAK,UBFG9,RUS,8712300.0,1.328268e+09,1.530318e+09,2012-02-03 11:26:34+00:00,2018-06-30 00:13:21+00:00
57386,0,NaN,273349630,2014-06-27 01:32:36+00:00,2014-06-28 01:21:20+00:00,58.224653,153.102242,25.157083,NaN,0.560717,...,1.403918e+09,273349630.0,ZODIAK,UBFG9,RUS,8712300.0,1.328268e+09,1.530318e+09,2012-02-03 11:26:34+00:00,2018-06-30 00:13:21+00:00
57387,0,NaN,273349630,2016-10-29 17:22:12+00:00,2016-10-30 21:12:23+00:00,57.452946,150.443527,29.134444,NaN,0.545175,...,1.477862e+09,273349630.0,ZODIAK,UBFG9,RUS,8712300.0,1.328268e+09,1.530318e+09,2012-02-03 11:26:34+00:00,2018-06-30 00:13:21+00:00
